<a href="https://colab.research.google.com/github/lol782/fake_new_detector/blob/main/Copy_of_fake_review_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn torch
!pip install lime


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import pandas as pd

# Load specific columns and rename
df_reduced = pd.read_csv("/content/fake reviews dataset.csv")[['label', 'text_']]
# Corrected typo: rename columns of df_reduced, not df
df_reduced = df_reduced.rename(columns={'text_': 'text'})

# Map labels and drop missing entries
df_reduced['label'] = df_reduced['label'].map({'OR': 0, 'CG': 1})  # OR = original, CG = computer-generated
df_reduced = df_reduced.dropna(subset=['label', 'text'])

# Check if the DataFrame is empty after dropping NaNs
print(f"Shape of df_reduced after dropping NaNs: {df_reduced.shape}")

# Reduce to 6000 rows - this will only work if df_reduced has rows
# If the shape printed above is (0, 2), the issue is with the data or mapping/dropping
df = df_reduced.sample(n=6000, random_state=42)
print(f"Shape of df_reduced after dropping NaNs: {df.shape}")


Shape of df_reduced after dropping NaNs: (40432, 2)
Shape of df_reduced after dropping NaNs: (6000, 2)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset



# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

# Create datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # Use "labels" key for Trainer
})
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

# Load the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Define training arguments (compatible with older versions of transformers)
training_args = TrainingArguments(
    output_dir="./bert_results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    # Remove or comment out problematic parameters
    # save_strategy="no",
    # logging_strategy="epoch",
    # evaluation_strategy="epoch"
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.156800


TrainOutput(global_step=600, training_loss=0.1365474514166514, metrics={'train_runtime': 854.6786, 'train_samples_per_second': 11.232, 'train_steps_per_second': 0.702, 'total_flos': 2525866131456000.0, 'train_loss': 0.1365474514166514, 'epoch': 2.0})

In [ ]:
!pip install huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository
from transformers import BertTokenizer, BertForSequenceClassification

# Set your model repo name (your-username/your-model-name)
model_repo = "lol3445/bert-email-cap-classifier"

# Save model and tokenizer locally first
model.save_pretrained("saved_bert_model")
tokenizer.save_pretrained("saved_bert_model")

# Push to the Hub
model.push_to_hub(model_repo)
tokenizer.push_to_hub(model_repo)


In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

Overwriting app.py


In [ ]:
import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from lime.lime_text import LimeTextExplainer
import pandas as pd
import numpy as np

# Load model/tokenizer
@st.cache_resource
def load_model():
    tokenizer = BertTokenizer.from_pretrained("lol3445/bert-email-cap-classifier")
    model = BertForSequenceClassification.from_pretrained("lol3445/bert-email-cap-classifier")
    return tokenizer, model

tokenizer, model = load_model()

def classify_email(text, tokenizer, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()
        confidence = probs[0][prediction].item()
    label = "Original" if prediction == 1 else "Computer-Generated"
    return label, confidence

def predict_proba(texts):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    if isinstance(texts, str):
        texts = [texts]
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
    return probs.cpu().numpy()

explainer = LimeTextExplainer(class_names=["Computer-Generated", "Original"])

st.set_page_config(page_title="Fake Review Detector", layout="centered")
st.title("🕵️‍♂️ Fake Review Detector")
st.write("Determine if a review is **Original** or **Computer-Generated** using BERT + LIME.")

user_input = st.text_area("✍️ Enter your Review Text:")
min_words_for_lime = 3

# Define label and confidence up front
label = None
confidence = None

if st.button("Classify"):
    if user_input.strip() == "":
        st.warning("Please enter some text.")
    else:
        label, confidence = classify_email(user_input, tokenizer, model)
        st.success(f"**Prediction: {label}**")
        st.info(f"Confidence Score: {confidence:.2%}")
        st.progress(confidence)

        # Feedback section
        st.markdown("### Was this prediction correct?")
        col1, col2 = st.columns(2)
        with col1:
            if st.button("👍 Yes, correct"):
                st.success("✅ Thanks for your feedback!")
                with open("feedback_log.txt", "a") as f:
                    f.write(f"👍 | Text: {user_input} | Predicted: {label} | Confidence: {confidence:.2%}\n")
        with col2:
            if st.button("👎 No, incorrect"):
                st.warning("⚠️ We'll use this to improve the model.")
                with open("feedback_log.txt", "a") as f:
                    f.write(f"👎 | Text: {user_input} | Predicted: {label} | Confidence: {confidence:.2%}\n")

# LIME Explanation
show_explanation = st.checkbox(
    "🔍 Show Explanation (LIME)",
    disabled=(len(user_input.strip().split()) < min_words_for_lime)
)

if show_explanation and user_input.strip():
    with st.spinner("Generating explanation..."):
        try:
            explanation = explainer.explain_instance(
                user_input,
                predict_proba,
                num_features=min(10, len(user_input.strip().split())),
                num_samples=100
            )
            lime_html = explanation.as_html()
            wrapped_html = f"""
            <div style="background-color: white; color: black; padding: 20px; border-radius: 10px;">
                <style>
                    .lime-explanation {{
                        background-color: white !important;
                        color: black !important;
                    }}
                    .lime-explanation table {{
                        background-color: white !important;
                        color: black !important;
                    }}
                    .lime-explanation td, .lime-explanation th {{
                        background-color: white !important;
                        color: black !important;
                        border: 1px solid #ccc !important;
                    }}
                    .lime-explanation .positive {{
                        background-color: #90EE90 !important;
                        color: black !important;
                    }}
                    .lime-explanation .negative {{
                        background-color: #FFB6C1 !important;
                        color: black !important;
                    }}
                </style>
                {lime_html}
            </div>
            """
            st.components.v1.html(wrapped_html, height=600, scrolling=True)
        except Exception as e:
            st.error(f"❌ Could not generate explanation: {str(e)}")
            st.info("💡 Try entering a longer, more descriptive review.")

# CSV Upload for Bulk Detection
st.markdown("---")
st.subheader("📁 Upload a CSV file for Bulk Detection")

uploaded_file = st.file_uploader("Upload a CSV file with a column named `text`", type=["csv"])

if uploaded_file:
    try:
        df_upload = pd.read_csv(uploaded_file)

        if "text" not in df_upload.columns:
            st.error("❌ The CSV must contain a column named `text`.")
        else:
            texts = df_upload["text"].astype(str).tolist()
            probs = predict_proba(texts)

            preds = np.argmax(probs, axis=1)
            confidences = probs[np.arange(len(preds)), preds]

            results_df = pd.DataFrame({
                "text": texts,
                "prediction": ["Original" if p == 1 else "Computer-Generated" for p in preds],
                "confidence": [f"{c:.2%}" for c in confidences]
            })

            st.success(f"✅ Processed {len(results_df)} reviews.")
            st.dataframe(results_df)

            csv_download = results_df.to_csv(index=False).encode("utf-8")
            st.download_button("⬇️ Download Results as CSV", csv_download, "results.csv", "text/csv")
    except Exception as e:
        st.error(f"⚠️ Error reading file: {e}")

if len(user_input.strip().split()) < min_words_for_lime:
    st.info(f"💡 Enter at least {min_words_for_lime} words to enable LIME explanation.")


2025-06-23 13:08:09.855 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:10.213 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-23 13:08:10.216 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:10.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:10.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:10.723 Thread 'Thread-8': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:10.724 Thread 'Thread-8': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:10.725 Thread 'Thread-8': missing 

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

2025-06-23 13:08:18.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.782 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-23 13:08:18.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.53.78.75


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.53.78.75:8501

⠙your url is: https://petite-plums-tease.loca.lt
2025-06-23 13:08:53.340705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750684133.383692    2778 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750684133.396044    2778 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
  Stopping...
^C
